# Defining custom problems
Many classic optimisation problems are [already implemented in PYMOO](https://www.pymoo.org/problems/index.html). However, you will want to implement your own problem at some point. Here's how to do that. 

In [1]:
import numpy as np
import autograd.numpy as anp
from pymoo.model.problem import Problem
from pymoo.util.misc import stack
from pymoo.model.problem import FunctionalProblem

Let's consider the following problem.

\begin{align}
\min \quad f_1(x) & = (x_1^2 + x_2^2) \\
\min \quad f_2(x) & = (x_1 - 1)^2 + x_2^2 \\
\textrm{s.t.} \quad 
g_1(x) & = 2(x_1−0.1)(x_1−0.9)/0.18 ≤ 0 \\
g_2(x) & = −20(x_1−0.4)(x_1−0.6)/4.8 ≤ 0 \\
−2 & ≤ x_1 ≤ 2  \\
−2 & ≤ x_2 ≤ 2
\end{align}

It consists of:
- 2 objective functions $f_1(x)$ and $f_2(x)$
- 2 design variables $x_1$ and $x_2$, both in the range $[−2,2]$
- 2 inequality contraints $g_1(x)$ and $g_2(x)$

There are 3 methods to define a custom problem in PYMOO: vectorized problem, elementwise problem and functional problem.

## Vectorized problem
The _evaluate method takes a **two-dimensional** NumPy array X with n rows and m columns as an input. Each row represents an individual, and each column an optimization variable. After doing the necessary calculations, the objective values must be added to the dictionary out with the key F and the constraints with key G.

In [2]:
class MyVectorizedProblem(Problem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, X, out, *args, **kwargs):
        f1 = X[:, 0]**2 + X[:, 1]**2
        f2 = (X[:, 0]-1)**2 + X[:, 1]**2

        g1 = 2*(X[:, 0]-0.1) * (X[:, 0]-0.9) / 0.18
        g2 = - 20*(X[:, 0]-0.4) * (X[:, 0]-0.6) / 4.8

        out["F"] = np.column_stack([f1, f2])
        out["G"] = np.column_stack([g1, g2])


vectorized_problem = MyVectorizedProblem()

## Elementwise problem
The _evaluate method takes a **one-dimensional** NumPy array x number of entries equal to n_var. This behavior is enabled by setting elementwise_evaluation=True while calling the super() method.

In [3]:
class MyElementwiseProblem(Problem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]),
                         elementwise_evaluation=True)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = x[0]**2 + x[1]**2
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


elementwise_problem = MyElementwiseProblem()

## Functional problem

In [4]:
objs = [
    lambda x: x[0]**2 + x[1]**2,
    lambda x: (x[0]-1)**2 + x[1]**2
]

constr_ieq = [
    lambda x: 2*(x[0]-0.1) * (x[0]-0.9) / 0.18,
    lambda x: - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8
]

functional_problem = FunctionalProblem(n_var=2,
                                       objs=objs,
                                       constr_ieq=constr_ieq,
                                       xl=np.array([-2,-2]),
                                       xu=np.array([2,2]))

## More vectorized problem examples

In [5]:
class SphereWithConstraint(Problem):

    def __init__(self):
        super().__init__(n_var=10, 
                         n_obj=1, 
                         n_constr=1, 
                         xl=0, 
                         xu=1)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = np.sum((x - 0.5) ** 2, axis=1)
        out["G"] = 0.1 - out["F"]
        
sphere_problem = SphereWithConstraint()

In [6]:
class MyVectorizedProblem2(Problem):

    def __init__(self, const_1=5, const_2=0.1):

        # define lower and upper bounds -  1d array with length equal to number of variable
        xl = -5 * anp.ones(10)
        xu = 5 * anp.ones(10)

        super().__init__(n_var=10, 
                         n_obj=1, 
                         n_constr=2, 
                         xl=xl, 
                         xu=xu, 
                         evaluation_of="auto")

        # store custom variables needed for evaluation
        self.const_1 = const_1
        self.const_2 = const_2

    def _evaluate(self, x, out, *args, **kwargs):
        f = anp.sum(anp.power(x, 2) - self.const_1 * anp.cos(2 * anp.pi * x), axis=1)
        g1 = (x[:, 0] + x[:, 1]) - self.const_2
        g2 = self.const_2 - (x[:, 2] + x[:, 3])

        out["F"] = f
        out["G"] = anp.column_stack([g1, g2])
        
        
vectorized_problem_2 = MyVectorizedProblem2()

## Another elementwise problem example

In [7]:
class MyElementwiseProblem2(Problem):

    def __init__(self, **kwargs):
        super().__init__(n_var=2, 
                         n_obj=1, 
                         elementwise_evaluation=True, 
                         **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = x.sum()


elementwise_problem_2 = MyElementwiseProblem2()

## Another functional problem example

In [8]:
objs = [
    lambda x: np.sum((x - 2) ** 2),
    lambda x: np.sum((x + 2) ** 2)
]

constr_ieq = [
    lambda x: np.sum((x - 1) ** 2)
]


functional_problem_2 = FunctionalProblem(10,
                            objs,
                            constr_ieq=constr_ieq,
                            xl=np.array([-10, -5, -10]),
                            xu=np.array([10, 5, 10])
                            )

# Problem evaluation
We can evaluate a problem with the evaluate() method. This method takes as input a two dimensional matrix where each row is a point to evaluate and each column a variable and returns a dictionary with the result of the evaluation.

In [9]:
# input vector: a random population of 3 individuals, each defined by 2 or 10 design variables.
X_2 = np.random.rand(3, 2)
X_10 = np.random.rand(3, 10)

In [10]:
vectorized_problem.evaluate(X_2, return_as_dictionary=True)

{'F': array([[0.81087827, 0.21912118],
        [1.5034484 , 0.78542261],
        [0.70667859, 0.82637445]]), 'CV': array([[0.        ],
        [0.        ],
        [0.02674261]]), 'G': array([[-0.80506541, -0.32310047],
        [-0.34566377, -0.49537609],
        [-1.73798028,  0.02674261]])}

In [11]:
elementwise_problem.evaluate(X_2, return_as_dictionary=True)

{'F': array([[0.81087827, 0.21912118],
        [1.5034484 , 0.78542261],
        [0.70667859, 0.82637445]]), 'CV': array([[0.        ],
        [0.        ],
        [0.02674261]]), 'G': array([[-0.80506541, -0.32310047],
        [-0.34566377, -0.49537609],
        [-1.73798028,  0.02674261]])}

In [12]:
functional_problem.evaluate(X_2, return_as_dictionary=True)

{'F': array([[0.81087827, 0.21912118],
        [1.5034484 , 0.78542261],
        [0.70667859, 0.82637445]]), 'CV': array([[0.        ],
        [0.        ],
        [0.02674261]]), 'G': array([[0.        , 0.        ],
        [0.        , 0.        ],
        [0.        , 0.02674261]])}

In [13]:
sphere_problem.evaluate(X_10, return_as_dictionary=True)

{'F': array([[1.15042437],
        [0.94046071],
        [1.02375656]]), 'CV': array([[0.],
        [0.],
        [0.]]), 'G': array([[-1.05042437],
        [-0.84046071],
        [-0.92375656]])}

In [14]:
# We can return more values in the output dictionary
vectorized_problem_2.evaluate(X_10, 
                              return_as_dictionary=True,
                              return_values_of=["F", "G", "CV", "feasible", "dF", "dG"])

{'F': array([[-8.07163185],
        [-2.29241499],
        [-5.20633783]]), 'G': array([[ 0.88579063, -1.10042268],
        [ 0.28220256, -0.40944205],
        [ 0.06113449, -0.91843568]]), 'CV': array([[0.88579063],
        [0.28220256],
        [0.06113449]]), 'feasible': array([[False],
        [False],
        [False]]), 'dF': array([[[ -7.25681455,   6.51259143, -25.46986727,  24.11557907,
           28.47885572,  17.03940501,  -4.80270106, -28.33370999,
           -8.16702513,  28.62121531]],
 
        [[ 29.36467993,  29.95899516,  31.91115334,  31.86247294,
            9.61746544,   2.02556659,  27.90385713,  -8.09201747,
          -18.88811863,  -6.58397957]],
 
        [[ 26.12460534,   1.51875465,  30.75106833, -27.56658396,
          -15.15107568,  -5.8975372 , -15.57973742,  31.53735039,
            6.29964495,  18.6469869 ]]]), 'dG': array([[[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
        [[ 1.,  

In [15]:
# in this problem, there is no constraints defined. The single objective function returns the sum of the 2 design variables.
# elementwise_problem_2.evaluate(X_2, return_as_dictionary=True)
elementwise_problem_2.evaluate([1, 0.5], return_as_dictionary=True)

{'F': array([1.5])}

In [16]:
functional_problem_2.evaluate(X_10, return_as_dictionary=True)

{'F': array([[21.50185041, 67.23138097],
        [26.20178287, 58.83825711],
        [26.89446125, 57.90591541]]), 'CV': array([[2.93423305],
        [4.36090143],
        [4.64732479]]), 'G': array([[2.93423305],
        [4.36090143],
        [4.64732479]])}